In [1]:
import getpass
import os

# input APIs and set up
os.environ["LANGSMITH_TRACING"] = "true"
if "LANGSMITH_API_KEY" not in os.environ:
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass(
        prompt="Enter your LangSmith API key (optional): "
    )
if "LANGSMITH_PROJECT" not in os.environ:
    os.environ["LANGSMITH_PROJECT"] = getpass.getpass(
        prompt='Enter your LangSmith Project Name (default = "default"): '
    )
    if not os.environ.get("LANGSMITH_PROJECT"):
        os.environ["LANGSMITH_PROJECT"] = "default"
if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-4o-mini", model_provider="openai")

Enter your LangSmith API key (optional):  ········
Enter your LangSmith Project Name (default = "default"):  ········
Enter API key for OpenAI:  ········


In [33]:
# using prompt templates
from langchain_core.prompts import ChatPromptTemplate

system_template = """You are a helpful assistant that transforms user text based on the following instructions:

Inputs:
- Translate: {translate}
- Target Language: {target_language}
- Summarize: {summary}
- Tone: {tone}

Behavior Rules:
1. If Translate is True and the detected language ≠ target language, translate the text.
2. If Translate is True and the detected language = target language, skip translation and explain why.
3. If Summarize is True, summarize the original text (not the translated version).
   - Apply the tone if provided:
     - "formal": use precise and structured language.
     - "humorous": be light, clever, and witty.
     - "casual": use relaxed and conversational language.
     - If no tone is given, use a neutral tone.
4. If both Translate and Summarize are False, return the original text and explain that no transformation was applied."""

In [38]:
translate = input("Translation? T/F")
target_language = input("Enter target language:")
summary = input("Summary? T/F") 
tone = input("Enter summary tone:")

Translation? T/F T
Enter target language: English
Summary? T/F T
Enter summary tone: witty


In [39]:
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

In [40]:
from langchain.chat_models import ChatOpenAI
prompt = prompt_template.invoke({"translate": translate, "target_language": target_language, "summary": summary, 
                                 "tone": tone, "text": """Once, there was a hare who was best friends with a tortoise. 
                                 The hare was very proud of how fast he could run, so one day, 
                                 he challenged the tortoise to a race. The tortoise agreed, even though everyone 
                                 thought he was way too slow to win. The race began, and the hare raced so fast that he 
                                 was far ahead of the tortoise. Feeling confident, the hare decided to take a nap 
                                 under a tree while the tortoise kept going, step by step. When the hare woke up, 
                                 he was shocked to see the tortoise crossing the finish line. The tortoise had won the race!"""})
response = model.invoke(prompt)
print(response.content)

Since both the translation and summarization requests are set to True, I'll go ahead and summarize the original text in a witty tone.

Here's the summary:

Once upon a time, a proud hare thought he could outpace his slowpoke tortoise buddy in a race. With confidence as high as his speed, he took a nap, just to wake up and find his friend plodding past the finish line. Turns out, slow and steady doesn’t just win the race; it also gives you plenty of time to catch up on some beauty sleep!
